In [1]:
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, './Preprocessing')
import numpy as np
import os
#os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
from imgDataGenerator import *
from preprocessing import *
import tensorflow as tf
import keras
from keras import applications
from keras.models import Sequential, Model, Input
from keras.layers import Conv2D, MaxPooling2D,MaxPooling3D, LSTM, Reshape, Permute, TimeDistributed, CuDNNLSTM,LeakyReLU, ConvLSTM2D
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization
from keras.models import model_from_json

from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.layers.merge import concatenate
from keras.optimizers import Adam
#suppress warnings
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
#Note for Test, Training, and Validation folders, they all need to have same categories
parentPath='/home/livelab/Desktop/NewResizedBinned'
checkAndGeneratePath(parentPath)
testPath='/home/livelab/Desktop/NewResizedBinned/Test'
trainPath='/home/livelab/Desktop/NewResizedBinned/Training'
validPath='/home/livelab/Desktop/NewResizedBinned/Validation'

trainingGen=imgDataGenerator(trainPath)
validGen=imgDataGenerator(validPath)

Length Check on Folder Succeed


In [3]:
inputS  = Input(shape=(None, 256, 256, 3))
    
first_ConvLSTM = ConvLSTM2D(filters=30, kernel_size=(3, 3)
                       , data_format='channels_last'
                       , recurrent_activation='hard_sigmoid'
                       , activation='tanh'
                       , padding='same', return_sequences=True,
                           dropout=0.2,
                           recurrent_dropout=0.2)(inputS)
first_BatchNormalization = BatchNormalization()(first_ConvLSTM)
first_Pooling = MaxPooling3D(pool_size=(2, 2, 2), padding='same', data_format='channels_last')(first_BatchNormalization)
    
second_ConvLSTM = ConvLSTM2D(filters=25, kernel_size=(3, 3)
                        , data_format='channels_last'
                        , padding='same', return_sequences=True,
                           dropout=0.2,
                           recurrent_dropout=0.2)(first_Pooling)
second_BatchNormalization = BatchNormalization()(second_ConvLSTM)
second_Pooling = MaxPooling3D(pool_size=( 2, 2,2), padding='same', data_format='channels_last')(second_BatchNormalization)
third_ConvLSTM = ConvLSTM2D(filters=15, kernel_size=(3, 3)
                        , data_format='channels_last'
                        , padding='same', return_sequences=True,
                           dropout=0.2,
                           recurrent_dropout=0.2)(second_Pooling)
third_BatchNormalization = BatchNormalization()(third_ConvLSTM)
third_Pooling = MaxPooling3D(pool_size=( 2, 2,2), padding='same', data_format='channels_last')(third_BatchNormalization)
fourth_ConvLSTM = ConvLSTM2D(filters=10, kernel_size=(3, 3)
                        , data_format='channels_last'
                        , padding='same', return_sequences=False,
                           dropout=0.2,
                           recurrent_dropout=0.2)(third_Pooling)
fourth_BatchNormalization = BatchNormalization()(fourth_ConvLSTM)
fourth_Pooling = MaxPooling2D(pool_size=( 2, 2), padding='same', data_format='channels_last')(fourth_BatchNormalization)
intermediateOut = Flatten()(fourth_Pooling)
#intermediateOut = Dense(32)(intermediateOut)
#intermediateOut = BatchNormalization()(intermediateOut)
#intermediateOut = LeakyReLU(alpha=0.1)(intermediateOut)
#intermediateOut = Dropout(0.4)(intermediateOut)
prediction = Dense(24, activation='softmax')(intermediateOut)
optmr = Adam(lr=0.003, decay = 5e-4)
model_final = Model(input = inputS, output = prediction)
model_final.compile(loss = "categorical_crossentropy", optimizer = optmr, metrics=["accuracy"])   

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [4]:
model_final.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, 256, 256, 3) 0         
_________________________________________________________________
conv_lst_m2d_1 (ConvLSTM2D)  (None, None, 256, 256, 30 35760     
_________________________________________________________________
batch_normalization_1 (Batch (None, None, 256, 256, 30 120       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, None, 128, 128, 30 0         
_________________________________________________________________
conv_lst_m2d_2 (ConvLSTM2D)  (None, None, 128, 128, 25 49600     
_________________________________________________________________
batch_normalization_2 (Batch (None, None, 128, 128, 25 100       
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, None, 64, 64, 25)  0         
__________

In [5]:
filepath = "CLSTM-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=True, mode='auto', period=1)
callbacks_list = [checkpoint]

In [6]:
with tf.device('GPU:0'):
    model_final.fit_generator(generator=trainingGen,
                    validation_data=validGen,
                    callbacks=callbacks_list,
                    use_multiprocessing=True,
                    epochs = 40,
                    workers=6)

Instructions for updating:
Use tf.cast instead.
Epoch 1/40
1286/1286 [==============================] - 621s 483ms/step - loss: 15.4946 - acc: 0.0373 - val_loss: 15.3190 - val_acc: 0.0294

Epoch 00001: val_acc improved from -inf to 0.02941, saving model to CLSTM-01-0.03.hdf5
Epoch 2/40
1286/1286 [==============================] - 613s 477ms/step - loss: 15.4914 - acc: 0.0389 - val_loss: 15.3165 - val_acc: 0.0294

Epoch 00002: val_acc did not improve from 0.02941
Epoch 3/40
1017/1286 [======================>.......] - ETA: 2:04 - loss: 15.5000 - acc: 0.0383

Process ForkPoolWorker-33:
Process ForkPoolWorker-32:
Process ForkPoolWorker-27:
Process ForkPoolWorker-26:
Process ForkPoolWorker-36:
Process ForkPoolWorker-35:
Process ForkPoolWorker-29:
Traceback (most recent call last):
Process ForkPoolWorker-30:
Traceback (most recent call last):
Process ForkPoolWorker-34:
Process ForkPoolWorker-31:
Process ForkPoolWorker-28:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/livelab/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/livelab/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/livelab/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/livelab/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._a

  File "/home/livelab/anaconda3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/home/livelab/anaconda3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
  File "/home/livelab/anaconda3/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
KeyboardInterrupt
  File "/home/livelab/anaconda3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt



Epoch 00002: val_acc did not improve from 0.02941
Epoch 3/40


KeyboardInterrupt: 

Process ForkPoolWorker-46:
Process ForkPoolWorker-45:
Process ForkPoolWorker-42:
Process ForkPoolWorker-44:
Process ForkPoolWorker-43:
Process ForkPoolWorker-37:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/livelab/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/livelab/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/livelab/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/livelab/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/livelab/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/home/livelab/anaconda3/lib

In [ ]:
#LSTM without Fully connected
#LSTM with branching

In [ ]:
x,y=trainingGen.__getitem__(30)

In [ ]:
np.argmax(y)

In [ ]:
len(x[0])